Задание 1

In [22]:
import numpy as np

payoffs = np.array([
    [[6, 8, 12, 5], [8, 12, 8, 7], [5, 11, 3, 9]],
    [[12, 6, 9, 2], [6, 13, 9, 5], [3, 5, 4, 9]],
    [[3, 4, 7, 7], [2, 14, 10, 9], [9, 4, 8, 5]],
    [[11, 2, 4, 14], [14, 6, 12, 11], [4, 7, 7, 3]],
    [[8, 11, 11, 7], [5, 11, 7, 6], [9, 5, 5, 7]]
])

def check_dominated(strategies):
    is_dominated = np.zeros(len(strategies), dtype=bool)
    for i, s1 in enumerate(strategies):
        for s2 in strategies:
            if (s2 >= s1).all() and not (s2 == s1).all():
                is_dominated[i] = True
                break
    return is_dominated

def find_dominated_indices(strategies):
    is_dominated = check_dominated(strategies)
    return np.where(is_dominated)[0]

dominated_indices_A = []
dominated_indices_B = []
for i in range(len(payoffs)):
    dominated_indices_A.append(find_dominated_indices(payoffs[i]))
    dominated_indices_B.append(find_dominated_indices(payoffs[i]))

new_payoffs_A = np.delete(payoffs, np.concatenate(dominated_indices_A), axis=1)
new_payoffs_B = np.delete(payoffs, np.concatenate(dominated_indices_B), axis=2)

mixed_strategy_A = np.zeros(len(new_payoffs_A[0]))
mixed_strategy_B = np.zeros(len(new_payoffs_B[0]))
for i in range(len(new_payoffs_A[0])):
    mixed_strategy_A[i] = 1 / len(new_payoffs_A[0])
for j in range(len(new_payoffs_B[0])):
    mixed_strategy_B[j] = 1 / len(new_payoffs_B[0])

game_price = np.sum(np.dot(mixed_strategy_A, new_payoffs_A[0])) 

print("Смешанные стратегии для игрока A:", mixed_strategy_A)
print("Смешанные стратегии для игрока B:", mixed_strategy_B)
print("Цена игры:", game_price)


Смешанные стратегии для игрока A: [0.5 0.5]
Смешанные стратегии для игрока B: [0.33333333 0.33333333 0.33333333]
Цена игры: 29.5


Задание 2

In [23]:
from scipy.optimize import minimize

threat_points = [40, 30, 15, 15, 15]
total_meat = 200

def objective(x):
    surplus = [x[i] - threat_points[i] for i in range(len(x))]
    product_of_surplus = 1
    for s in surplus:
        if s < 0:
            return float('inf')  
        product_of_surplus *= s
    return -product_of_surplus

constraints = (
    {'type': 'eq', 'fun': lambda x: sum(x) - total_meat},  
    {'type': 'ineq', 'fun': lambda x: x[0] - threat_points[0]},  
    {'type': 'ineq', 'fun': lambda x: x[1] - threat_points[1]},  
    {'type': 'ineq', 'fun': lambda x: x[2] - threat_points[2]},  
    {'type': 'ineq', 'fun': lambda x: x[3] - threat_points[3]},  
    {'type': 'ineq', 'fun': lambda x: x[4] - threat_points[4]}   
)

initial_guess = [40, 30, 15, 15, 15]

result = minimize(objective, initial_guess, constraints=constraints)

if result.success:
    allocation = result.x
    print("Справедливое распределение мяса:")
    for i, meat in enumerate(allocation):
        print(f"Охотник {i+1}: {meat:.2f} кг")
else:
    print("Не удалось найти справедливое распределение")


Справедливое распределение мяса:
Охотник 1: 57.00 кг
Охотник 2: 47.00 кг
Охотник 3: 32.00 кг
Охотник 4: 32.00 кг
Охотник 5: 32.00 кг


Задание 3

In [24]:
from scipy.optimize import minimize

points_of_threat = [10, 8, 14]
initial_values = [20, 20, 30]  

def nash_function(X):
    X1, X2, X3 = X
    if X1 < points_of_threat[0] or X2 < points_of_threat[1] or X3 < points_of_threat[2]:
        return float('-inf') 
    return -(X1 - points_of_threat[0]) * (X2 - points_of_threat[1]) * (X3 - points_of_threat[2])

def constraint(X):
    return 70 - sum(X)

bounds = [(points_of_threat[0], None), (points_of_threat[1], None), (points_of_threat[2], None)]

constraints = {'type': 'eq', 'fun': constraint}

result = minimize(nash_function, initial_values, method='SLSQP', bounds=bounds, constraints=constraints)

if result.success:
    max_nash_value = -result.fun
    optimal_X = result.x
    print(f"Максимальное значение функции Нэша: {round(max_nash_value)}")
    print(f"Оптимальные значения X1, X2, X3: {optimal_X}")
else:
    print("Не удалось найти решение")



Максимальное значение функции Нэша: 2032
Оптимальные значения X1, X2, X3: [22.6666199  20.66649954 26.66688057]
